In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

# EX 10.2

In [ ]:
z <- scan("DAT/tourist.txt")
forecast::tsdisplay(z)


- 이분산성, 추세와 계절성분을 모두 가지고 있다.

- 변수변환을 통한 분산안정화 부터 진행


## (1) 분산안정화 : 변수변환

In [ ]:
log_z = log(z)
sqrt_z = sqrt(z)
boxcox_z = forecast::BoxCox(z,lambda= forecast::BoxCox.lambda(z))

In [ ]:
forecast::BoxCox.lambda(z)

In [ ]:
par(mfrow=c(2,2))
plot.ts(z, main = "original")
plot.ts(log_z, main = 'log')
plot.ts(sqrt_z, main = 'sqrt')
plot.ts(boxcox_z, main = 'Boxcox')

In [ ]:
t = 1:length(z)
lmtest::bptest(lm(z~t)) #H0 : 등분산이다
lmtest::bptest(lm(log_z~t))
lmtest::bptest(lm(sqrt_z~t))
lmtest::bptest(lm(boxcox_z~t))

- $\lambda = 0.597$을 이용한 Boxcox 변환 수행

In [ ]:
forecast::tsdisplay(boxcox_z)

In [ ]:
- 계절성분과 추세가 존재하므로 우선 계절차분을 통한 계절성분 제거

## (2) 계절차분

In [ ]:
lag12_boxcox_z = diff(boxcox_z, lag=12)
forecast::tsdisplay(lag12_boxcox_z)

- 계절차분을 통해 제절추세는 제거 되었음

- 결정적 추세는 없는 것으로 보이나, ACF가 조금은 천천히 감소하는 경향이 보이기 때문에 단위근 검정을 통해
차분이 필요한가에 대한 여부 판정


## (3) 단위근 검정

In [ ]:
##단위근 검정 : H0 : 단위근이 있다.
fUnitRoots::adfTest(lag12_boxcox_z, lags = 1, type = "c")


- 유의확률이 아주 크지는 않지만, 유의수준 $\alpha=0.05$하에서 귀무가설을 기각하지 못하기 때문에 차분을 하기로
함


In [ ]:
diff_lag12_boxcox_z = diff(lag12_boxcox_z)
forecast::tsdisplay(diff_lag12_boxcox_z)


In [ ]:
## 단위근검정 H0 : phi=1
fUnitRoots::adfTest(diff_lag12_boxcox_z, lags = 1, type = "nc")


- 차분한 후의 자료에 대해서는 더이상 차분이 필요하지 않음


## (4) 모형식별

In [ ]:
forecast::tsdisplay(diff_lag12_boxcox_z)

- ACF는 1차와 12차에서 절단 형태이고, PACF는 1,2차와 12,13 시차 등에서 유의하게 나왔다.

- ACF의 차수가 더 낮으므로 MA(1)과 계절형 MA(1)모형을 적합할 수 있다.

- diff_lag12_boxcox_z = : $ARIMA(0, 0, 1)(0, 0, 1)_{12}$

- lag12_boxcox_z = $ARIMA(0,1,1)(0,0,1)_{12}$

- Boxcox_z = $ARIMA(0,1,1)(0,1,1)_{12}$

## (5) 모형적합

In [ ]:
fit1 = arima(boxcox_z, order = c(0,1,1),
     seasonal = list(order = c(0,1,1), period=12))
fit1

In [ ]:
lmtest::coeftest(fit1)

$(1 − B)(1 − B^{12})BoxCox(Z) = (1 − 0.5307B)(1 − 0.6370B^{12})ε_t, ε+t ∼ WN(0, 4831)$

## (6) 잔차분석

In [ ]:
forecast::checkresiduals(fit1)

- 잔차의 평균은 0이다

In [ ]:
t.test(resid(fit1))

- 등분산성을 만족하고, ACF가 모든 시차에서 유의하지 않으므로 WN라고 할 수 있다.

`-` 포투맨투검정

In [ ]:
#모형 적합도 검정 : H0 : rho1=...=rho_k=0
Box.test(resid(fit1), lag=1, type = "Ljung-Box")
Box.test(resid(fit1), lag=6, type = "Ljung-Box")
Box.test(resid(fit1), lag=12, type = "Ljung-Box")

- 모든 시차에 대해 유의확률값이 크기 때문에 WN라고 할 수 있다.

In [1]:
# 잔차의 포트맨토 검정 ## H0 : rho1=...=rho_k=0
#portes::LjungBox(fit1, lags=c(6,12,18,24))

## (7) 과대적합

- boxcox_z = $ARIMA(0,1,1)(0,1,1)_{12}$

- $ARIMA(1,1,1)(0,1,1)_{12}, ARIMA(0,1,2)(0,1,1)_{12}$

In [ ]:
fit2 <- arima(boxcox_z, order = c(1,1,1),
     seasonal = list(order = c(0,1,1), period=12))
fit2

In [ ]:
lmtest::coeftest(fit2)

In [ ]:
fit3 <- arima(boxcox_z, order = c(0,1,2),
     seasonal = list(order = c(0,1,1), period=12))
fit3


In [ ]:
lmtest::coeftest(fit3)

In [ ]:
fit1

- $ARIMA(0,1,2)(0,1,1)_{12}$모형 적합 결과가 더 좋기 때문에 최종 모형으로 선택

In [ ]:
forecast::checkresiduals(fit3)

In [ ]:
#모형 적합도 검정 : H0 : rho1=...=rho_k=0
Box.test(resid(fit3), lag=1, type = "Ljung-Box")
Box.test(resid(fit3), lag=6, type = "Ljung-Box")
Box.test(resid(fit3), lag=12, type = "Ljung-Box")

- 잔차검정 결과도 이상이 없으므로 fit3을 최종모형으로 선택

## (8) auto.arima

In [ ]:
forecast::auto.arima(ts(boxcox_z, frequency=12),
                     test = "adf",
                     seasonal = TRUE, trace = T)

In [ ]:
fit4 <- forecast::auto.arima(ts(boxcox_z, frequency=12),
                             test = "adf",
                             seasonal = TRUE)


In [ ]:
lmtest::coeftest(fit4)

- 최종선택 모형에 비해 aic나 분산이 조금 작지만 추가된 계수가 유의하지 않으므로 기존 모형 유지

## (9) 예측

In [ ]:
fore_fit <- forecast::forecast(fit3, 24)

In [ ]:
fore_fit

In [ ]:
plot(fore_fit)

In [ ]:
astsa::sarima.for(boxcox_z, 24, p=0,d=1,q=2,P=0, D=1, Q=1, S=12)

- BoxCox 역변환을 해야함

In [ ]:
forecast_z = forecast::InvBoxCox(fore_fit$mean, lambda=forecast::BoxCox.lambda(z))
forecast_z


In [ ]:
forecast_z_upper95 = forecast::InvBoxCox(fore_fit$upper[,2], lambda=forecast::BoxCox.lambda(z))
forecast_z_lower95 = forecast::InvBoxCox(fore_fit$lower[,2], lambda=forecast::BoxCox.lambda(z))


In [ ]:
plot(1:(length(z)+24), c(z, forecast_z_upper95), type='n', xlab='t', ylab='z')
lines(1:length(z),z)
#lines((length(z)+1) : (length(z)+24), forecast_z, col='red')
#lines((length(z)+1) : (length(z)+24), forecast::InvBoxCox(fore_fit$lower[,2], lambda=forecast::BoxCox.la
#lines((length(z)+1) : (length(z)+24), forecast::InvBoxCox(fore_fit$upper[,2], lambda=forecast::BoxCox.la


# Simulation

In [2]:
#astsa::sarima.sim(ar = NULL, d = 0, ma = NULL, sar = NULL, D = 0, sma = NULL, S = NULL,
# n = 500, rand.gen = rnorm, innov = NULL, burnin = NA, t0 = 0, ...)
#sarima::sim_sarima(model=list(ar,iorder,ma, sar, siorder, sma, nseasons, sigma2),
# n = NA, rand.gen = rnorm, n.start = NA, x, eps,
# xcenter = NULL, xintercept = NULL, ...)


In [ ]:
### ARIMA(p,d,q)(P,D,Q)_s
x <- sarima::sim_sarima(n=100, n.start=12,
                         model=list(ar = 0.7, siorder=1, nseasons=12, sigma2 = 1)) # SMA(1)
forecast::tsdisplay(x)

In [ ]:
x <- astsa::sarima.sim(ar = 0.5, d = 1, ma = 0.1, sar = 0.1, D = 1, sma = 0, S = 12, n = 100)
forecast::tsdisplay(x)